# DFloat11 Model needs minimum L4 GPU

In [1]:
# @title 1. Set up environment

# Clone the repository
print("Cloning BagelUI fork repository...")
!git clone https://github.com/dasjoms/BagelUI.git

# Navigate into the BAGEL directory
%cd BagelUI

# --- Dependency Installation ---
# Colab typically has some versions of torch/torchvision pre-installed.
print("\nUninstalling pre-existing torch/torchvision...")
!pip uninstall torch torchvision torchaudio -y

print("\nInstalling pip requirements from requirements.txt...")
# Install dependencies from requirements.txt
!pip install -r requirements.txt

# Force numpy upgrade
!pip install "numpy>=1.25.0,<2.0.0" --upgrade

print("\nInstalling flash_attn==2.5.8...")
# Install flash_attn. (This may take a while)
!pip install flash_attn==2.5.8 --no-build-isolation -vvv

!pip install dfloat11
print("\nAll dependencies installed!")

Cloning BagelUI fork repository...
Cloning into 'BagelUI'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 362 (delta 129), reused 90 (delta 90), pack-reused 165 (from 2)
Receiving objects: 100% (362/362), 2.19 MiB | 27.67 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/content/BagelUI

Uninstalling pre-existing torch/torchvision...
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124

Installing pip requirements from requirements.txt...
Ignoring triton-windows: markers 'sys_platform == "win32"' don't match your environment
  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 101.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 11.0.0 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.10.1 which is incompatible.
dask-expr 1.1.21 requires pyarrow>=14.0.1, but you have pyarrow 11.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.0 which is incompatible.
pylibcudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine 

In [3]:
# @title 2. Download Regular BAGEL-7B and DFloat11
from huggingface_hub import snapshot_download
import os

print("Starting model downloads...")

# Ensure the base 'models' directory exists
os.makedirs("models", exist_ok=True)

# --- Download BAGEL-7B-MoT (original bfloat16 model) ---
save_dir_orig = "models/BAGEL-7B-MoT"
repo_id_orig = "ByteDance-Seed/BAGEL-7B-MoT"
cache_dir_orig = os.path.join(save_dir_orig, "cache") # Hugging Face cache for this model

print(f"\nDownloading {repo_id_orig}...")
# Create save directory for the original model
os.makedirs(save_dir_orig, exist_ok=True)

snapshot_download(
    cache_dir=cache_dir_orig,
    local_dir=save_dir_orig,
    repo_id=repo_id_orig,
    local_dir_use_symlinks=False, # Copy files rather than symlink
    resume_download=True,         # Resume if interrupted
    # Specify patterns to download only essential files
    allow_patterns=["*.json", "*.safetensors", "*.bin", "*.py", "*.md", "*.txt", "*.model"],
)

print(f"Model {repo_id_orig} downloaded to: {os.path.abspath(save_dir_orig)}")


# --- Download BAGEL-7B-MoT-DF11 (DFloat11 compressed model) ---
save_dir_df11 = "models/BAGEL-7B-MoT-DF11"
repo_id_df11 = "DFloat11/BAGEL-7B-MoT-DF11"
cache_dir_df11 = os.path.join(save_dir_df11, "cache") # Hugging Face cache for this model

print(f"\nDownloading {repo_id_df11}...")
# Create save directory for the DFloat11 model
os.makedirs(save_dir_df11, exist_ok=True)

snapshot_download(
    cache_dir=cache_dir_df11,
    local_dir=save_dir_df11,
    repo_id=repo_id_df11,
    local_dir_use_symlinks=False,
    resume_download=True,
    # Specify patterns. Added "vae/*" to ensure the VAE subfolder is downloaded
    allow_patterns=["*.json", "*.safetensors", "*.bin", "*.py", "*.md", "*.txt", "*.model", "vae/*"],
)

print(f"Model {repo_id_df11} downloaded to: {os.path.abspath(save_dir_df11)}")

print("\nAll model downloads complete and ready for use!")

Starting model downloads...



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a loc

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

ae.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

ae.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vit_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

vit_model.safetensors:   0%|          | 0.00/550M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

Model DFloat11/BAGEL-7B-MoT-DF11 downloaded to: /content/BagelUI/models/BAGEL-7B-MoT-DF11

All model downloads complete and ready for use!


In [4]:
# @title 3. Start the Gradio WebUI!
# Remove '--mode 3' if using A100 GPU
# The modes are: 1 (No Quant), 2 (4-Bit Quant) and 3 (8-Bit Quant)
# Change model in 'Models' tab

!python app.py --share --mode 3

2025-06-03 17:33:44.665569: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 17:33:45.365555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748972025.682567   24736 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748972025.765291   24736 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 17:33:46.417085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr